In [ ]:
from helper_functions.shared_functions import check_if_s3_key_exists, run_sql_in_db, get_db_engine
from helper_functions.viz_classes import database
import os
import codecs
import csv
import boto3
import json
import pandas as pd
from datetime import datetime
from io import StringIO
from sqlalchemy.exc import DataError
from sqlalchemy.types import Text

fim_version = "4.3.11.0"
fim_folder = f"fim_{fim_version.replace('.', '_')}"
hand_datasets = f"{fim_folder}/hand_datasets"
qa_datasets = f"{fim_folder}/qa_datasets"
fim_bucket = "hydrovis-ti-deployment-us-east-1"
fim_crosswalk = os.path.join(hand_datasets, "crosswalk_table.csv")
pipeline_arn = "arn:aws:states:us-east-1:526904826677:stateMachine:viz_pipeline_ti"

In [ ]:
os.environ['VIZ_DB_USERNAME'] = ''
os.environ['VIZ_DB_PASSWORD'] = ''
ti_access_key = ''
ti_secret_key = ''
ti_token = ''

In [ ]:
s3_client = boto3.client("s3")
client = boto3.client('stepfunctions')
viz_engine = get_db_engine('viz')

<h2>1 - UPDATE VLAB REPO WITH NEW FIM VERSION</h2>

<h2>2 - UPLOAD FIM4 HYDRO ID/FEATURE ID CROSSWALK</h2>

In [ ]:
print(f"Getting column name from {fim_crosswalk}")
data = s3_client.get_object(Bucket=fim_bucket, Key=fim_crosswalk)
d_reader = csv.DictReader(codecs.getreader("utf-8")(data["Body"]))
headers = d_reader.fieldnames

header_str = "("
for header in headers:
    header_str += header
    if header in ['HydroID', 'LakeID', 'feature_id']:
        header_str += ' integer,'
    elif header in ['BranchID']:
        header_str += ' bigint,'
    else:
        header_str += ' TEXT,'
header_str = header_str[:-1] + ")"


db = database(db_type="viz")
with db.get_db_connection() as conn, conn.cursor() as cur:
    print(f"Deleting/Creating derived.fim4_featureid_crosswalk using columns {header_str}")
    sql = f"DROP TABLE IF EXISTS derived.fim4_featureid_crosswalk; CREATE TABLE derived.fim4_featureid_crosswalk {header_str};"
    cur.execute(sql)
    conn.commit()

    print(f"Importing {fim_crosswalk} to derived.fim4_featureid_crosswalk")
    sql = f"""
        SELECT aws_s3.table_import_from_s3(
           'derived.fim4_featureid_crosswalk',
           '', 
           '(format csv, HEADER true)',
           (SELECT aws_commons.create_s3_uri(
               '{fim_bucket}',
               '{fim_crosswalk}',
               'us-east-1'
                ) AS s3_uri
            ),
            aws_commons.create_aws_credentials('{ti_access_key}', '{ti_secret_key}', '{ti_token}')
           );
        """
    cur.execute(sql)
    conn.commit()

    print(f"Adding fim_version column to derived.fim4_featureid_crosswalk")
    sql = f"ALTER TABLE derived.fim4_featureid_crosswalk ADD COLUMN fim_version text DEFAULT '{fim_version}';"
    cur.execute(sql)
    conn.commit()

    print(f"Renaming columns in derived.fim4_featureid_crosswalk")
    sql = f"""
        ALTER TABLE derived.fim4_featureid_crosswalk RENAME COLUMN HydroID TO hydro_id;
        ALTER TABLE derived.fim4_featureid_crosswalk RENAME COLUMN LakeID TO lake_id;
        ALTER TABLE derived.fim4_featureid_crosswalk RENAME COLUMN BranchID  TO branch_id;
    """
    cur.execute(sql)
    conn.commit()

    print(f"Adding feature id index to derived.fim4_featureid_crosswalk")
    sql = f"CREATE INDEX fim4_crosswalk_feature_id ON derived.fim4_featureid_crosswalk USING btree (feature_id)"
    cur.execute(sql)
    conn.commit()

    print(f"Adding hydro id index to derived.fim4_featureid_crosswalk")
    sql = f"CREATE INDEX fim4_crosswalk_hydro_id ON derived.fim4_featureid_crosswalk USING btree (hydro_id)"
    cur.execute(sql)
    conn.commit()

print(f"Successully updated derived.fim4_featureid_crosswalk")

<h2>3 - UPDATE FIM HAND PROCESSING LAMBDA WITH NEW FIM VERSION</h2>

<h2>4 - UPDATE FIM DATA PREP LAMBDA WITH NEW FIM VERSION</h2>

<h2>5 - UPDATE INITIALIZE PIPELINE LAMBDA TO RUN ONLY AEP FIM SERVICE. UNCOMMENT 2 FIM CONFIGS AT A TIME TO RUN THE AEP CONFIGURATION. CHECK FOR STEP FUNCTION FINISHING BEFORE STARTING NEW ONE.</h2>

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_nwm_aep_inundation_extent_library",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "rf_2_inundation",
          "target_table": "aep_fim.rf_2_inundation",
          "fim_type": "hand",
          "sql_file": "rf_2_inundation"
        }
      ],
      "services": [
        "static_nwm_aep_inundation_extent_library_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_aep_2_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_nwm_aep_inundation_extent_library",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "rf_5_inundation",
          "target_table": "aep_fim.rf_5_inundation",
          "fim_type": "hand",
          "sql_file": "rf_5_inundation"
        }
      ],
      "services": [
        "static_nwm_aep_inundation_extent_library_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_aep_5_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_nwm_aep_inundation_extent_library",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "rf_10_inundation",
          "target_table": "aep_fim.rf_10_inundation",
          "fim_type": "hand",
          "sql_file": "rf_10_inundation"
        }
      ],
      "services": [
        "static_nwm_aep_inundation_extent_library_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_aep_10_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_nwm_aep_inundation_extent_library",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "rf_25_inundation",
          "target_table": "aep_fim.rf_25_inundation",
          "fim_type": "hand",
          "sql_file": "rf_25_inundation"
        }
      ],
      "services": [
        "static_nwm_aep_inundation_extent_library_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_aep_25_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_nwm_aep_inundation_extent_library",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "rf_50_inundation",
          "target_table": "aep_fim.rf_50_inundation",
          "fim_type": "hand",
          "sql_file": "rf_50_inundation"
        }
      ],
      "services": [
        "static_nwm_aep_inundation_extent_library_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_aep_50_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_nwm_aep_inundation_extent_library",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "rf_high_water_inundation",
          "target_table": "aep_fim.rf_high_water_inundation",
          "fim_type": "hand",
          "sql_file": "rf_high_water_inundation"
        }
      ],
      "services": [
        "static_nwm_aep_inundation_extent_library_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_aep_hw_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

<h2>6 - RUN CATCHMENT WORKFLOWS 2 CONFIGS AT A TIME. CHECK FOR STEP FUNCTION FINISHING BEFORE STARTING NEW ONE</h2>

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_hand_catchments_0_branches",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "branch_0_catchments",
          "target_table": "fim_catchments.branch_0_catchments",
          "fim_type": "hand",
          "sql_file": "branch_0_catchments"
        }
      ],
      "services": [
        "static_hand_catchments_0_branches_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    },
    {
      "product": "static_hand_catchments_0_branches_hi",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "branch_0_catchments_hi",
          "target_table": "fim_catchments.branch_0_catchments_hi",
          "fim_type": "hand",
          "sql_file": "branch_0_catchments_hi"
        }
      ],
      "services": [
        "static_hand_catchments_0_branches_hi_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    },
    {
      "product": "static_hand_catchments_0_branches_prvi",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "branch_0_catchments_prvi",
          "target_table": "fim_catchments.branch_0_catchments_prvi",
          "fim_type": "hand",
          "sql_file": "branch_0_catchments_prvi"
        }
      ],
      "services": [
        "static_hand_catchments_0_branches_prvi_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_0_catchments_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

In [ ]:
pipeline_input = {
  "configuration": "reference",
  "job_type": "auto",
  "data_type": "channel",
  "keep_raw": False,
  "reference_time": datetime.now().strftime('%Y-%m-%d 00:00:00'),
  "configuration_data_flow": {
    "db_max_flows": [],
    "db_ingest_groups": [],
    "lambda_max_flows": []
  },
  "pipeline_products": [
    {
      "product": "static_hand_catchments_gms_branches",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "branch_gms_catchments",
          "target_table": "fim_catchments.branch_gms_catchments",
          "fim_type": "hand",
          "sql_file": "branch_gms_catchments"
        }
      ],
      "services": [
        "static_hand_catchments_gms_branches_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    },
    {
      "product": "static_hand_catchments_gms_branches_hi",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "branch_gms_catchments_hi",
          "target_table": "fim_catchments.branch_gms_catchments_hi",
          "fim_type": "hand",
          "sql_file": "branch_gms_catchments_hi"
        }
      ],
      "services": [
        "static_hand_catchments_gms_branches_hi_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    },
    {
      "product": "static_hand_catchments_gms_branches_prvi",
      "configuration": "reference",
      "product_type": "fim",
      "run": True,
      "fim_configs": [
        {
          "name": "branch_gms_catchments_prvi",
          "target_table": "fim_catchments.branch_gms_catchments_prvi",
          "fim_type": "hand",
          "sql_file": "branch_gms_catchments_prvi"
        }
      ],
      "services": [
        "static_hand_catchments_gms_branches_prvi_noaa"
      ],
      "raster_outputs": {
        "output_bucket": "",
        "output_raster_workspaces": []
      },
      "postprocess_sql": [],
      "product_summaries": [],
      "lambda_max_flow_dependent": False
    }
  ],
  "sql_rename_dict": {}
}

client.start_execution(
    stateMachineArn = pipeline_arn,
    name = f"sagemaker_gms_catchments_{datetime.now().strftime('%Y%m%dT%H%M')}",
    input= json.dumps(pipeline_input)
)

<h2>6 - UPDATE RATING CURVES IN DB</h2>

In [ ]:
sql = '''
DROP TABLE IF EXISTS derived.hydrotable;
DROP TABLE IF EXISTS derived.usgs_elev_table;

SELECT distinct LPAD(huc8::text, 8, '0') as huc8 FROM derived.featureid_huc_crosswalk WHERE huc8 is not null;
'''
df = run_sql_in_db(sql)

col_names = ['HydroID', 'feature_id', 'NextDownID', 'order_', 'Number of Cells',
       'SurfaceArea (m2)', 'BedArea (m2)', 'TopWidth (m)', 'LENGTHKM',
       'AREASQKM', 'WettedPerimeter (m)', 'HydraulicRadius (m)',
       'WetArea (m2)', 'Volume (m3)', 'SLOPE', 'ManningN', 'stage',
       'default_discharge_cms', 'default_Volume (m3)', 'default_WetArea (m2)',
       'default_HydraulicRadius (m)', 'default_ManningN',
       'precalb_discharge_cms', 'calb_coef_spatial', 'HUC', 'LakeID',
       'subdiv_applied', 'channel_n', 'overbank_n', 'subdiv_discharge_cms',
       'last_updated', 'submitter', 'calb_coef_usgs', 'obs_source',
       'calb_coef_final', 'calb_applied', 'discharge_cms']

usecols = ['HydroID', 'feature_id', 'stage', 'discharge_cms']

paginator = s3_client.get_paginator('list_objects')
operation_parameters = {'Bucket': fim_bucket,
                        'Prefix': f'{hand_datasets}/',
                        'Delimiter': '/'}
page_iterator = paginator.paginate(**operation_parameters)
page_count = 0
for page in page_iterator:
    page_count += 1
    prefix_objects = page['CommonPrefixes']
    for i, prefix_obj in enumerate(prefix_objects):
        print(f"Processing {i+1} of {len(prefix_objects)} on page {page_count}")
        branch_prefix = f'{prefix_obj.get("Prefix")}branches/0/'
#         ## UNCOMMENT FOR ALL BRANCHES - NOT JUST 0
#         huc_branches_prefix = f'{prefix_obj.get("Prefix")}branches/'
#         branches_result = s3_client.list_objects(Bucket=fim_bucket, Prefix=huc_branches_prefix, Delimiter='/')
#         branch_prefix_objects = branches_result.get('CommonPrefixes')
#         for i, branch_prefix_obj in enumerate(branch_prefix_objects):
#             branch_prefix = branch_prefix_obj['Prefix']
#         ## END UNCOMMENT
        ## [UN]INDENT FROM HERE TO THE END IF [COMMENTED]UNCOMMENTED
        branch_files_result = s3_client.list_objects(Bucket=fim_bucket, Prefix=branch_prefix, Delimiter='/')
        hydro_table_key = None
        usgs_elev_table_key = None
        for content_obj in branch_files_result.get('Contents'):
            branch_file_prefix = content_obj['Key']
            if 'hydroTable' in branch_file_prefix:
                hydro_table_key = branch_file_prefix
            elif 'usgs_elev_table.csv' in branch_file_prefix:
                usgs_elev_table_key = branch_file_prefix

        if hydro_table_key and usgs_elev_table_key:
#             print(f"Found usgs_elev_table and hydroTable in {branch_prefix}")
            try:
#                 print("...Fetching csvs...")
                uet = s3_client.get_object(Bucket=fim_bucket, Key=usgs_elev_table_key)['Body']
                ht = s3_client.get_object(Bucket=fim_bucket, Key=hydro_table_key)['Body']
#                 print("...Reading with pandas...")
                uet_df = pd.read_csv(uet)
                ht_df = pd.read_csv(ht, header=0, names=col_names, usecols=usecols)
#                 print('...Writing to db...')
                uet_df['fim_version'] = fim_version
                try:
                    drop_indicies = []
                    for index, row in uet_df.iterrows():
                        if 'HydroID' in row and row['HydroID']:
                            iter_hydro_id = row['HydroID']
#                             print(f"Subsetting hydrotable where HydroID == {iter_hydro_id}")
                            ht_df_iter = ht_df[ht_df['HydroID']==iter_hydro_id]
                            if ht_df_iter.empty:
                                drop_indicies.append(index)
                                continue
                            ht_df_iter['fim_version'] = fim_version
                            ht_df_iter.to_sql(con=viz_engine, schema='derived', name='hydrotable', index=False, if_exists='append')
                        else:
                            drop_indicies.append(index)
                    mod_uet_df = uet_df.drop(drop_indicies)
                    mod_uet_df.to_sql(con=viz_engine, dtype={"location_id": Text(), "nws_data_huc": Text()}, schema='derived', name='usgs_elev_table', index=False, if_exists='append')
                except Exception as e:
                    print('******************************************')
                    print(f'Error encountered on {branch_file_prefix}')
                    print(e)
                    print('******************************************')       
            except Exception as e:
                raise e
                print(f'Fetch failed: {e}')


In [ ]:
sql = '''
DROP TABLE IF EXISTS derived.hydrotable_staggered;
SELECT
    et.location_id,
    ht.feature_id,
    (stage + et.dem_adj_elevation) * 3.28084 as elevation_ft,
    LEAD((stage + et.dem_adj_elevation) * 3.28084) OVER (PARTITION BY ht.feature_id ORDER BY ht.feature_id, stage) as next_elevation_ft,
    discharge_cms * 35.3147 as discharge_cfs,
    LEAD(discharge_cms * 35.3147) OVER (PARTITION BY ht.feature_id ORDER BY ht.feature_id, stage) as next_discharge_cfs
INTO derived.hydrotable_staggered
FROM derived.hydrotable AS ht
JOIN derived.usgs_elev_table AS et ON ht."HydroID" = et."HydroID" AND et.location_id IS NOT NULL;
'''

run_sql_in_db(sql)

In [ ]:
usgs_curves_key = f'{qa_datasets}/usgs_rating_curves.csv'
obj_body = s3_client.get_object(Bucket=fim_bucket, Key=usgs_curves_key)['Body']
df = pd.read_csv(obj_body)
df.to_sql(con=viz_engine, schema='derived', name='usgs_rating_curves', index=False, if_exists='replace', chunksize=150000, method='multi')

In [ ]:
sql = '''
DROP TABLE IF EXISTS derived.usgs_rating_curves_staggered;
SELECT 
    location_id,
    flow as discharge_cfs, 
    LEAD(flow) OVER (PARTITION BY location_id ORDER BY location_id, stage) as next_discharge_cfs,
    stage,
    navd88_datum,
    elevation_navd88 as elevation_ft,
    LEAD(elevation_navd88) OVER (PARTITION BY location_id ORDER BY location_id, stage) as next_elevation_ft
INTO derived.usgs_rating_curves_staggered
FROM derived.usgs_rating_curves;
'''

run_sql_in_db(sql)

<h2>7 - UPDATE SRC SKILL METRICS IN DB</h2>

In [ ]:
src_stats_key = f'{fim_folder}/qa_datasets/agg_nwm_recurr_flow_elev_stats_location_id.csv'
obj_body = s3_client.get_object(Bucket=fim_bucket, Key=src_stats_key)['Body']
df = pd.read_csv(obj_body)
# df['fim_version'] = fim_version
print(df.to_dict('records'))
# df.to_sql(con=viz_engine, schema='derived', name='src_skill', index=False, if_exists='replace')

<h2>8 - UPDATE FIM PERFORMANCE METRICS IN DB</h2>

In [ ]:
import os
from helper_functions.shared_functions import *
import boto3
import geopandas as gpd
import pandas as pd

# os.environ['EGIS_DB_HOST'] = ''
os.environ['EGIS_DB_HOST'] = ''
os.environ['EGIS_DB_DATABASE'] = ''
os.environ['EGIS_DB_USERNAME'] = ''
os.environ['EGIS_DB_PASSWORD'] = ''

db_type = "egis"
db_engine = get_db_engine(db_type)
db_type = "egis"
db_engine = get_db_engine(db_type)

s3 = boto3.client('s3')

# Define bucket and parent directories.
bucket = "hydrovis-ti-deployment-us-east-1"
parent_directory = "qc_fim_data"
local_download_parent_directory = f'brad_data/qc_fim_data'

file_handles = ['fim_performance_points.csv', 'fim_performance_polys.csv', 'fim_performance_catchments.csv']

for file_handle in file_handles:
    # Define path to file to download and its local download path, the download.
    filename = f"{fim_folder}/qa_datasets/{file_handle}"
    print(filename)
    local_download_path = os.path.join(local_download_parent_directory, f'{file_handle}')
    print(f"--> Downloading {fim_bucket}/{filename} to {local_download_path}")
    s3.download_file(fim_bucket, filename, local_download_path)
    
    #  -- Open file and reformat -- #
    print("Reading file...")
    df = pd.read_csv(local_download_path)
    print("File read.")
    # Rename headers.
    if file_handle == 'fim_performance_points.csv':
        df = df.rename(columns={'Unnamed: 0': 'oid', 'geometry': 'geom'})
    else:
        df = df.rename(columns={'Unnamed: 0': 'oid', 'geometry': 'geom', 'huc':'huc8'})
        
    # Convert all field names to lowercase (needed for ArcGIS Pro).
    df.columns= df.columns.str.lower()

    # Enforce data types on df before loading in DB (TODO: need to create special cases for each layer).
    if file_handle == 'fim_performance_points.csv':
        df = df.astype({'huc': 'str'})
    else:
        df = df.astype({'huc8': 'str'})
    df = df.fillna(0)
    try:
        df = df.astype({'feature_id': 'int'})
        df = df.astype({'feature_id': 'str'})
    except KeyError:  # If there is no feature_id field
        pass
    try:
        df = df.astype({'nwm_seg': 'int'})
        df = df.astype({'nwm_seg': 'str'})
    except KeyError:  # If there is no nwm_seg field
        pass
    try:
        df = df.astype({'usgs_gage': 'int'})
        df = df.astype({'usgs_gage': 'str'})
    except KeyError:  # If there is no usgs_gage field
        pass
        
    # zfill HUC8 field.
    if file_handle == 'fim_performance_points.csv':
        df['huc'] = df['huc'].apply(lambda x: x.zfill(8))
    else:
        df['huc8'] = df['huc8'].apply(lambda x: x.zfill(8))
 
    # Upload df to database.
    stripped_layer_name = file_handle.replace(".csv", "")
    table_name = "reference." + stripped_layer_name
    print("Loading data into DB...")
    
    # Chunk load data into DB
    if file_handle in ['fim_performance_catchments.csv']:
        print("Chunk loading...")
        # Create list of df chunks
        n = 100000  #chunk row size
        list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
        
        # Load the first chunk into the DB as a new table
        first_chunk_df = list_df[0]
        print(first_chunk_df.shape[0])
        load_df_into_db(table_name, db_engine, first_chunk_df, epsg=3857, drop_first=True)
        
        # Load remaining chunks into newly created table
        for remaining_chunk in list_df[1:]:
            print(remaining_chunk.shape[0])
            load_df_into_db(table_name, db_engine, remaining_chunk, epsg=3857, drop_first=False)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')
        
    else:
        load_df_into_db(table_name, db_engine, df, epsg=3857, drop_first=True)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')
    
    if file_handle == 'fim_performance_catchments.csv':
        print("Making valid...")
        make_valid_sql = f"""

        ALTER TABLE reference.fim_performance_catchments
          ADD COLUMN geom_invalid geometry
          DEFAULT NULL;

        UPDATE reference.fim_performance_catchments
          SET geom = ST_MakeValid(geom),
              geom_invalid = geom
          WHERE NOT ST_IsValid(geom);
          """

        db_engine.execute(make_valid_sql)

<h2>9 - UPDATE STAGE-BASED CATFIM DATA IN DB</h2>

In [ ]:
import os
from helper_functions.shared_functions import *
import boto3
import geopandas as gpd
import pandas as pd

# os.environ['EGIS_DB_HOST'] = ''
os.environ['EGIS_DB_HOST'] = ''
os.environ['EGIS_DB_DATABASE'] = ''
os.environ['EGIS_DB_USERNAME'] = ''
os.environ['EGIS_DB_PASSWORD'] = ''

db_type = "egis"
db_engine = get_db_engine(db_type)

s3 = boto3.client('s3')

# Define bucket and parent directories.
bucket = "hydrovis-ti-deployment-us-east-1"
parent_directory = "qc_fim_data"
local_download_parent_directory = f'brad_data/qc_fim_data'

file_handles = ['stage_based_catfim_sites.csv', 'stage_based_catfim.csv']

for file_handle in file_handles:
    # Define path to file to download and its local download path, the download.
    filename = f"{fim_folder}/qa_datasets/{file_handle}"
    print(filename)
    local_download_path = os.path.join(local_download_parent_directory, f'{file_handle}')
    print(f"--> Downloading {fim_bucket}/{filename} to {local_download_path}")
    s3.download_file(fim_bucket, filename, local_download_path)
    
    #  -- Open file and reformat -- #
    print("Reading file...")
    df = pd.read_csv(local_download_path)
    print("File read.")
    
    # Rename headers.
    df = df.rename(columns={'Unnamed: 0': 'oid', 'geometry': 'geom', 'huc':'huc8'})
        
    # Convert all field names to lowercase (needed for ArcGIS Pro).
    df.columns= df.columns.str.lower()

    # Remove sites that are in derived.ahps_restricted_sites
    restricted_sites_df = get_db_values("derived.ahps_restricted_sites", "*")
    restricted_dict = restricted_sites_df.to_dict('records')

    # Change 'mapped' to 'no' if sites are present in restricted_sites_df
    for site in restricted_dict:
        nws_lid = site['nws_lid'].lower()
        if "sites" in file_handle:
            df.loc[df.ahps_lid==nws_lid, 'mapped'] = 'no'
            df.loc[df.ahps_lid==nws_lid, 'status'] = site['restricted_reason']
        else:
            df.loc[df.ahps_lid==nws_lid, 'viz'] = 'no'
            df = df[df['viz']=='yes']  # Subset df to only sites desired for mapping
            
    for sea_level_site in ['qutg1', 'augg1', 'baxg1', 'lamf1', 'adlg1', 'hrag1', 'stng1']:
        if "sites" in file_handle:
            df.loc[df.ahps_lid==sea_level_site, 'mapped'] = 'no'
            df.loc[df.ahps_lid==sea_level_site, 'status'] = 'Stage thresholds seem to be based on sea level and not channel thalweg'
        else:
            df.loc[df.ahps_lid==sea_level_site, 'viz'] = 'no'
            df = df[df['viz']=='yes']  # Subset df to only sites desired for mapping
    
    # Enforce data types on df before loading in DB (TODO: need to create special cases for each layer).
    df = df.astype({'huc8': 'str'})
    df = df.fillna(0)
    
    try:
        df = df.astype({'feature_id': 'int'})
        df = df.astype({'feature_id': 'str'})
    except KeyError:  # If there is no feature_id field
        pass
    try:
        df = df.astype({'nwm_seg': 'int'})
        df = df.astype({'nwm_seg': 'str'})
    except KeyError:  # If there is no nwm_seg field
        pass
    try:
        df = df.astype({'usgs_gage': 'int'})
        df = df.astype({'usgs_gage': 'str'})
    except KeyError:  # If there is no usgs_gage field
        pass
        
    # zfill HUC8 field.
    df['huc8'] = df['huc8'].apply(lambda x: x.zfill(8))
    
    if file_handle in ['stage_based_catfim_sites.csv']:
        df = df.astype({'nws_data_rfc_forecast_point': 'str'})
        df = df.astype({'nws_data_rfc_defined_fcst_point': 'str'})
        df = df.astype({'nws_data_riverpoint': 'str'})
        
    # Upload df to database.
    stripped_layer_name = file_handle.replace(".csv", "")
    table_name = "reference." + stripped_layer_name
    print("Loading data into DB...")
    
    print("Dataframe shape")
    print(df.shape[0])
    
    # Chunk load data into DB
    if file_handle in ['stage_based_catfim.csv']:
        print("Chunk loading...")
        # Create list of df chunks
        n = 1000  #chunk row size
        list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
        
        # Load the first chunk into the DB as a new table
        first_chunk_df = list_df[0]
        print(first_chunk_df.shape[0])
        load_df_into_db(table_name, db_engine, first_chunk_df, epsg=3857, drop_first=True)
        
        # Load remaining chunks into newly created table
        for remaining_chunk in list_df[1:]:
            print(remaining_chunk.shape[0])
            load_df_into_db(table_name, db_engine, remaining_chunk, epsg=3857, drop_first=False)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')
        
    else:
        load_df_into_db(table_name, db_engine, df, epsg=3857, drop_first=True)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')

<h2>10 - UPDATE FLOW-BASED CATFIM DATA IN DB</h2>

In [ ]:
import os
from helper_functions.shared_functions import *
import boto3
import geopandas as gpd
import pandas as pd

# os.environ['EGIS_DB_HOST'] = ''
os.environ['EGIS_DB_HOST'] = ''
os.environ['EGIS_DB_DATABASE'] = ''
os.environ['EGIS_DB_USERNAME'] = ''
os.environ['EGIS_DB_PASSWORD'] = ''

db_type = "egis"
db_engine = get_db_engine(db_type)

s3 = boto3.client('s3')

# Define bucket and parent directories.
bucket = "hydrovis-ti-deployment-us-east-1"
parent_directory = "qc_fim_data"
local_download_parent_directory = f'brad_data/qc_fim_data'

file_handles = ['flow_based_catfim.csv', 'flow_based_catfim_sites.csv']

for file_handle in file_handles:
    # Define path to file to download and its local download path, the download.
    filename = f"{fim_folder}/qa_datasets/{file_handle}"
    print(filename)
    local_download_path = os.path.join(local_download_parent_directory, f'{file_handle}')
    print(f"--> Downloading {fim_bucket}/{filename} to {local_download_path}")
    s3.download_file(fim_bucket, filename, local_download_path)
    
    #  -- Open file and reformat -- #
    print("Reading file...")
    df = pd.read_csv(local_download_path)
    print("File read.")
    # Rename headers.
    df = df.rename(columns={'Unnamed: 0': 'oid', 'geometry': 'geom', 'huc':'huc8'})
        
    # Convert all field names to lowercase (needed for ArcGIS Pro).
    df.columns= df.columns.str.lower()

    # Remove sites that are in derived.ahps_restricted_sites
    restricted_sites_df = get_db_values("derived.ahps_restricted_sites", "*")
    restricted_dict = restricted_sites_df.to_dict('records')

    # Change 'mapped' to 'no' if sites are present in restricted_sites_df
    for site in restricted_dict:
        nws_lid = site['nws_lid'].lower()
        if "sites" in file_handle:
            #print(True)
            #print(nws_lid)
            df.loc[df.ahps_lid==nws_lid, 'mapped'] = 'no'
            df.loc[df.ahps_lid==nws_lid, 'status'] = site['restricted_reason']
            #print(df.loc[df.ahps_lid==nws_lid]['status'])
        else:
            df.loc[df.ahps_lid==nws_lid, 'viz'] = 'no'
            df = df[df['viz']=='yes']
    
    # Enforce data types on df before loading in DB (TODO: need to create special cases for each layer).
    df = df.astype({'huc8': 'str'})
    df = df.fillna(0)
    try:
        df = df.astype({'feature_id': 'int'})
        df = df.astype({'feature_id': 'str'})
    except KeyError:  # If there is no feature_id field
        pass
    try:
        df = df.astype({'nwm_seg': 'int'})
        df = df.astype({'nwm_seg': 'str'})
    except KeyError:  # If there is no nwm_seg field
        pass
    try:
        df = df.astype({'usgs_gage': 'int'})
        df = df.astype({'usgs_gage': 'str'})
    except KeyError:  # If there is no usgs_gage field
        pass
        
    # zfill HUC8 field.
    df['huc8'] = df['huc8'].apply(lambda x: x.zfill(8))
    
    if file_handle in ['flow_based_catfim_sites.csv']:
        df = df.astype({'nws_data_rfc_forecast_point': 'str'})
        df = df.astype({'nws_data_rfc_defined_fcst_point': 'str'})
        df = df.astype({'nws_data_riverpoint': 'str'})
        
    # Upload df to database.
    stripped_layer_name = file_handle.replace(".csv", "")
    table_name = "reference." + stripped_layer_name
    print("Loading data into DB...")
    
    print("Dataframe shape")
    print(df.shape[0])
    
    # Chunk load data into DB
    if file_handle in ['flow_based_catfim.csv']:
        print("Chunk loading...")
        # Create list of df chunks
        n = 10000  #chunk row size
        list_df = [df[i:i+n] for i in range(0,df.shape[0],n)]
        
        # Load the first chunk into the DB as a new table
        first_chunk_df = list_df[0]
        print(first_chunk_df.shape[0])
        load_df_into_db(table_name, db_engine, first_chunk_df, epsg=3857, drop_first=True)
        
        # Load remaining chunks into newly created table
        for remaining_chunk in list_df[1:]:
            print(remaining_chunk.shape[0])
            load_df_into_db(table_name, db_engine, remaining_chunk, epsg=3857, drop_first=False)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')
        
    else:
        load_df_into_db(table_name, db_engine, df, epsg=3857, drop_first=True)
        db_engine.execute(f'CREATE INDEX ON reference.{stripped_layer_name} USING GIST (geom);')